In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
pwd

'/content'

In [5]:
cd /content/gdrive/My Drive/test/Mr. Cooper/

/content/gdrive/My Drive/test/Mr. Cooper


In [6]:
!pip install contractions
!pip install transformers

     |████████████████████████████████| 284 kB 12.9 MB/s 
     |████████████████████████████████| 321 kB 43.4 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85442 sha256=6d3bf4d83099afc32345a915afe12c2564315fa972ae308415e5a64979e0b285
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
     |████████████████████████████████| 2.9 MB 12.6 MB/s 
     |████████████████████████████████| 895 kB 43.2 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 44.4 MB/s 
     |████████████████████████████████| 3.3 MB 42.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# ML Case Study
### Objective
##### To be written

In [7]:
# Text Normalization
import re
import spacy
import unicodedata
import contractions

# Data Wrangling
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

# Data Visualizations
# import matplotlib.pyplot as plt
# from matplotlib import rcParams
# import seaborn as sns
import plotly.express as px

# Data Preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

# Neural Network Model
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW


# Model Validation
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

In [8]:
# specify GPU
device = torch.device("cuda")

# Label Enconding
le = LabelEncoder()

# Loading Spacy Model
nlp = spacy.load('en_core_web_sm')

In [9]:
device

device(type='cuda')

# Data Loading

In [10]:
# reading data from .csv file
data_df = pd.read_csv("data/Context.csv")
data_df.head(3)

,Text,Context/Topic
0,The eternal mystique of Goldman Sachs,Politics
1,Either you don't care enough to actually tell ...,Love
2,I am such an IDIOT.,Heavy Emotion


In [11]:
# Each feature type info
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31386 entries, 0 to 31385
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Text           31386 non-null  object
 1   Context/Topic  31386 non-null  object
dtypes: object(2)
memory usage: 490.5+ KB


In [12]:
# Size of total dataset
data_df.shape

(31386, 2)

# Data Cleaning

In [ ]:
# Checking if any Null value is present
data_df.isna().sum()

Text             0
Context/Topic    0
dtype: int64

In [ ]:
# Checking if any Duplicate value is present
data_df.duplicated().sum()

0

In [13]:
# Removing URLs
def remove_urls(text):
    text = re.sub(r'http\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'www\S+', ' ', text, flags=re.MULTILINE)
    return (text)


# Removing bracketed words
def remove_bracket_words(text):
    text = re.sub("[\(\[].*?[\)\]]", " ", text)
    return text


# Normalizing contracted words
def remove_contraction_word(text):
    text = " ".join([contractions.fix(i) for i in text.split() if len(i) > 0 and "@" not in i])
    return text


# Removing ascii characters and symbols words
def remove_ascii_character(text):
    text = re.sub('[^.,:a-zA-Z0-9 \n\.]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


# Normalizing accented words
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


# Cleaning Pipeline
def cleaning_pipeline(text):
    cleaned_text = remove_urls(text)
    cleaned_text = remove_bracket_words(cleaned_text)
    cleaned_text = remove_contraction_word(cleaned_text)
    cleaned_text = remove_ascii_character(cleaned_text)
    cleaned_text = remove_accented_chars(text)

    return cleaned_text


In [14]:
cleaned_text = []
count = 1
for text in data_df['Text']:
    cleaned_text.append(cleaning_pipeline(text))
    if count%10000 == 0:
        print("Records processed:\t", count)
    # break
    count = count + 1
print("Records processed:\t", count)

Records processed:	 10000
Records processed:	 20000
Records processed:	 30000
Records processed:	 31387


In [15]:
data_df = data_df.assign(cleaned_text=cleaned_text)

We can see that there is no null value and duplicate value is present

# Exploratory Data Analysis

In [16]:
# Desctiption of each features
for col in data_df.columns:
    print("\n***********************")
    print("Columns:", col)
    print(data_df[col].describe())



***********************
Columns: Text
count                                                 31386
unique                                                31386
top       Getting children used to the idea of discussin...
freq                                                      1
Name: Text, dtype: object

***********************
Columns: Context/Topic
count             31386
unique               11
top       Heavy Emotion
freq               3674
Name: Context/Topic, dtype: object

***********************
Columns: cleaned_text
count     31386
unique    31317
top            
freq         29
Name: cleaned_text, dtype: object


In [ ]:
# profile = ProfileReport(data_df, title="Pandas Profiling Report")
# profile

#### Total Number of variables are 2, including target variable

Among that one variables is text and another one is category of each text
1. Text is textual content is present
2. Context/Topic is categorical, 11 distinct value is present, which is representing the text content

In [17]:
data_df['Context/Topic'].value_counts()

Heavy Emotion    3674
Religion         3466
Love             3229
Self             3105
Compliment       3061
Animals          2622
Health           2595
Education        2534
Joke             2476
Science          2428
Politics         2196
Name: Context/Topic, dtype: int64

In [22]:
length = []
for text in data_df['cleaned_text']:
    length.append(len(text.split()))
    
data_df['length'] = length

In [23]:
data_df['context_topic'] = le.fit_transform(data_df['Context/Topic'])
data_df.head(3)

,Text,Context/Topic,cleaned_text,length,context_topic
0,The eternal mystique of Goldman Sachs,Politics,The eternal mystique of Goldman Sachs,6,7
1,Either you don't care enough to actually tell ...,Love,Either you don't care enough to actually tell ...,93,6
2,I am such an IDIOT.,Heavy Emotion,I am such an IDIOT.,5,4


In [24]:
fig = px.histogram(data_df, x="length")
fig.show()

In [25]:
data_df['length'].describe()

count    31386.000000
mean        24.330816
std         43.529349
min          0.000000
25%          6.000000
50%         11.000000
75%         24.000000
max       1132.000000
Name: length, dtype: float64

In [26]:
topic_avg_leg = []

for each_g in data_df.groupby(['Context/Topic']):
    topic_avg_leg.append([each_g[0], np.mean(each_g[1]['length'])])
#     break
    

In [27]:
df = pd.DataFrame(topic_avg_leg, columns=['topic', 'length'])

fig = px.bar(df.sort_values(['length']), x='topic', y='length',
             color='length',
             height=400)
fig.show()

In [28]:
data_mod_df = data_df[(data_df['length'] > 0) & (data_df['length'] <= 50)]

In [29]:
data_mod_df.shape

(27797, 5)

In [30]:
data_mod_df['Context/Topic'].value_counts()

Religion         3039
Compliment       2882
Self             2877
Heavy Emotion    2769
Animals          2577
Health           2471
Love             2464
Science          2395
Education        2233
Politics         2189
Joke             1901
Name: Context/Topic, dtype: int64

In [31]:
# check class distribution
data_mod_df['Context/Topic'].value_counts(normalize = True)

Religion         0.109328
Compliment       0.103680
Self             0.103500
Heavy Emotion    0.099615
Animals          0.092708
Health           0.088894
Love             0.088643
Science          0.086160
Education        0.080332
Politics         0.078750
Joke             0.068389
Name: Context/Topic, dtype: float64

# Split train dataset into train, validation and test sets

In [32]:
train_text, temp_text, train_labels, temp_labels = train_test_split(data_mod_df['Text'], data_mod_df['context_topic'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=data_mod_df['context_topic'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [33]:
print("Size of Independent Training Data", train_text.shape)
print("Size of Dependent Training Data", train_labels.shape)
print("Size of Independent Test Data", test_text.shape)
print("Size of Dependent Training Data", test_labels.shape)
print("Size of Independent Validation Data", val_text.shape)
print("Size of Dependent Validation Data", val_labels.shape)


Size of Independent Training Data (19457,)
Size of Dependent Training Data (19457,)
Size of Independent Test Data (4170,)
Size of Dependent Training Data (4170,)
Size of Independent Validation Data (4170,)
Size of Dependent Validation Data (4170,)


# Import BERT Model and BERT Tokenizer

In [34]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

# Tokenization

In [35]:
# get length of all the messages in the train set
seq_len = [len(i.split()) for i in train_text]

pd.Series(seq_len).hist()

In [36]:
max_seq_len = 52

In [37]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# Convert Integer Sequences to Tensors

In [38]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

# Create DataLoaders

In [39]:
#define a batch size
batch_size = 128

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# Freeze BERT Parameters

In [40]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

# Define Model Architecture

In [41]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,11)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [42]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [43]:
# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

# Find Class Weights

In [44]:
#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print(class_wts)

[0.98049788 0.87695497 1.1316815  1.02243826 0.91270288 1.32893928
 1.02540184 1.15458106 0.83160234 1.05538078 0.87826126]


In [45]:
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

# Fine-Tune BERT

In [46]:
# function to train the model
def train():
  
    model.train()

    total_loss, total_accuracy = 0, 0

    # empty list to save model predictions
    total_preds=[]

    # iterate over batches
    for step,batch in enumerate(train_dataloader):
    
        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]
    
        sent_id, mask, labels = batch

        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

        # append the model predictions
        total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds


In [47]:
# function for evaluating the model
def evaluate():
  
    print("\nEvaluating...")

    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_preds = []

    # iterate over batches
    for step,batch in enumerate(val_dataloader):
    
        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            
            # Calculate elapsed time in minutes.
            # elapsed = format_time(time.time() - t0)
            # elapsed = time.time() - t0
                
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
        # model predictions
        preds = model(sent_id, mask)

        # compute the validation loss between actual and predicted values
        loss = cross_entropy(preds,labels)

        total_loss = total_loss + loss.item()

        preds = preds.detach().cpu().numpy()

        total_preds.append(preds)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds
    
# evaluate()

# Start Model Training

In [48]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
  Batch    50  of    153.
  Batch   100  of    153.
  Batch   150  of    153.

Evaluating...

Training Loss: 1.967
Validation Loss: 0.056

 Epoch 2 / 10
  Batch    50  of    153.
  Batch   100  of    153.
  Batch   150  of    153.

Evaluating...

Training Loss: 1.569
Validation Loss: 0.049

 Epoch 3 / 10
  Batch    50  of    153.
  Batch   100  of    153.
  Batch   150  of    153.

Evaluating...

Training Loss: 1.449
Validation Loss: 0.048

 Epoch 4 / 10
  Batch    50  of    153.
  Batch   100  of    153.
  Batch   150  of    153.

Evaluating...

Training Loss: 1.373
Validation Loss: 0.044

 Epoch 5 / 10
  Batch    50  of    153.
  Batch   100  of    153.
  Batch   150  of    153.

Evaluating...

Training Loss: 1.339
Validation Loss: 0.045

 Epoch 6 / 10
  Batch    50  of    153.
  Batch   100  of    153.
  Batch   150  of    153.

Evaluating...

Training Loss: 1.286
Validation Loss: 0.045

 Epoch 7 / 10
  Batch    50  of    153.
  Batch   100  of    153.
  Batch   150  

# Load Saved Model

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
# model.load_state_dict(torch.load(path))
device_ = torch.device('cpu')
model.load_state_dict(torch.load(path, map_location=device_))

<All keys matched successfully>

# Get Predictions for Test Data

In [50]:
# get predictions for test data
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    # preds = model(test_seq, test_mask)
    # preds = preds.detach().cpu().numpy()

RuntimeError: ignored

In [ ]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

In [1]:
# device
import sklearn
sklearn.__version__

'0.22.2.post1'